In [ ]:
SUBS = {
    "0": "₀",
    "1": "₁",
    "2": "₂",
    "3": "₃",
    "4": "₄",
    "5": "₅",
    "6": "₆",
    "7": "₇",
    "8": "₈",
    "9": "₉",
}


def subscript(num):
    return "".join(SUBS[d] for d in str(num))


def varname(i):
    return f"x{subscript(i)}"


def indent_block(s, indent):
    if not s:
        return indent
    return "\n".join(indent + line for line in s.splitlines())


def box_table(headers, rows):
    widths = [max(len(h), *(len(r[i]) for r in rows)) for i, h in enumerate(headers)]

    def hline(left, mid, right, fill="─"):
        s = left
        for i, w in enumerate(widths):
            s += fill * (w + 2)
            s += mid if i < len(widths) - 1 else right
        return s

    top = hline("┌", "┬", "┐")
    header_sep = hline("├", "┼", "┤")
    bottom = hline("└", "┴", "┘")
    out = [top]
    hdr = "│"
    for i, h in enumerate(headers):
        hdr += " " + h.center(widths[i]) + " │"
    out.append(hdr)
    out.append(header_sep)
    for r in rows:
        row = "│"
        for i, c in enumerate(r):
            row += " " + c.center(widths[i]) + " │"
        out.append(row)
    out.append(bottom)
    return "\n".join(out)


def bits_of(num, n):
    return [(num >> (n - 1 - i)) & 1 for i in range(n)]


def num_from_bits(bits):
    val = 0
    for b in bits:
        val = (val << 1) | (1 if b else 0)
    return val


def compute_truth_set(minterms):
    if not minterms:
        return 1, set()
    n = max(minterms).bit_length()
    return n, set(minterms)


def cofactor_set(ones, n, var_index, value):
    res = set()
    for m in ones:
        bit = (m >> (n - 1 - var_index)) & 1
        if bit == value:
            bits = bits_of(m, n)
            del bits[var_index]
            res.add(num_from_bits(bits))
    return res


def derivative_set(ones, n, var_index):
    f0 = cofactor_set(ones, n, var_index, 0)
    f1 = cofactor_set(ones, n, var_index, 1)
    return f0 ^ f1


def print_truth_table(n, ones, vars_list, indent=""):
    headers = [varname(idx) for idx in vars_list] + ["f"]
    rows = []
    for m in range(1 << n):
        rows.append([str(b) for b in bits_of(m, n)] + ["1" if m in ones else "0"])
    table = box_table(headers, rows)
    print(indent_block(table, indent))


def print_derivatives(n, ones, vars_list, indent=""):
    for i in range(n):
        other_vars = [varname(vars_list[j]) for j in range(n) if j != i]
        headers = other_vars + [
            f"f{subscript(vars_list[i])}₀",
            f"f{subscript(vars_list[i])}₁",
            f"∂f/∂{varname(vars_list[i])}",
        ]
        rows = []
        for a in range(1 << max(0, n - 1)):
            other_bits = bits_of(a, n - 1) if n - 1 > 0 else []
            bits0 = other_bits.copy()
            bits0.insert(i, 0)
            bits1 = other_bits.copy()
            bits1.insert(i, 1)
            m0 = num_from_bits(bits0)
            m1 = num_from_bits(bits1)
            f0 = "1" if m0 in ones else "0"
            f1 = "1" if m1 in ones else "0"
            xor = "1" if f0 != f1 else "0"
            rows.append([str(b) for b in other_bits] + [f0, f1, xor])
        table = box_table(headers, rows)
        print(indent_block(table, indent))
        print(
            indent
            + f"Вес ∂f/∂{varname(vars_list[i])} = {len(derivative_set(ones, n, i))}\n"
        )


def rec(ones, n, vars_list, depth=0, type=-1):
    indent = "  " * depth
    total = 1 << n

    sub = ""

    if type >= 0:
        sub = subscript(type)

    print(
        f"{indent}--- Узел уровня {depth} f{sub}({', '.join(varname(v) for v in vars_list)}) ---"
    )
    print_truth_table(n, ones, vars_list, indent)
    print()
    print(indent + "Кофакторы и производные:")
    print_derivatives(n, ones, vars_list, indent)
    if len(ones) == 0:
        print(indent + "Функция = 0 (константа)\n")
        return "0"
    if len(ones) == total:
        print(indent + "Функция = 1 (константа)\n")
        return "1"
    if n == 1:
        if ones == {1}:
            print(indent + f"Функция = {varname(vars_list[0])}\n")
            return varname(vars_list[0])
        if ones == {0}:
            print(indent + f"Функция = ¬{varname(vars_list[0])}\n")
            return f"¬{varname(vars_list[0])}"
    best_i = None
    best_w = None
    for i in range(n):
        w = len(derivative_set(ones, n, i))
        if best_w is None or w > best_w:
            best_w = w
            best_i = i
    i = best_i
    global_idx = vars_list[i]
    print(
        indent
        + f"Выбрана переменная {varname(global_idx)} (локальный индекс {i}, вес производной = {best_w})"
    )
    f1 = cofactor_set(ones, n, i, 1)
    f0 = cofactor_set(ones, n, i, 0)
    print(indent + "Разложение:")
    print(
        indent + f"f = f(..., {varname(global_idx)}=1, ...) & {varname(global_idx)} ∨ "
        f"f(..., {varname(global_idx)}=0, ...) & ¬{varname(global_idx)}\n"
    )
    new_vars = vars_list[:i] + vars_list[i + 1 :]
    expr1 = rec(f1, n - 1, new_vars, depth + 1, 1)
    expr0 = rec(f0, n - 1, new_vars, depth + 1, 0)
    expr = f"({expr1} & {varname(global_idx)}) ∨ ({expr0} & ¬{varname(global_idx)})"
    print(indent + f"-> Результат на уровне {depth}: {expr}\n")
    return expr


def main(minterms):
    n, ones = compute_truth_set(minterms)
    vars_list = list(range(1, n + 1))
    expr = rec(ones, n, vars_list, 0)
    print("Итоговое выражение:")
    print(expr)


if __name__ == "__main__":
    main(list(map(int, input("Введите термы через пробел: ").split())))

Входные минтермы: [2, 3, 5, 7, 10, 11, 14]
Определено число переменных: n = 4

--- Узел уровня 0 f(x₁, x₂, x₃, x₄) ---
┌────┬────┬────┬────┬───┐
│ x₁ │ x₂ │ x₃ │ x₄ │ f │
├────┼────┼────┼────┼───┤
│ 0  │ 0  │ 0  │ 0  │ 0 │
│ 0  │ 0  │ 0  │ 1  │ 0 │
│ 0  │ 0  │ 1  │ 0  │ 1 │
│ 0  │ 0  │ 1  │ 1  │ 1 │
│ 0  │ 1  │ 0  │ 0  │ 0 │
│ 0  │ 1  │ 0  │ 1  │ 1 │
│ 0  │ 1  │ 1  │ 0  │ 0 │
│ 0  │ 1  │ 1  │ 1  │ 1 │
│ 1  │ 0  │ 0  │ 0  │ 0 │
│ 1  │ 0  │ 0  │ 1  │ 0 │
│ 1  │ 0  │ 1  │ 0  │ 1 │
│ 1  │ 0  │ 1  │ 1  │ 1 │
│ 1  │ 1  │ 0  │ 0  │ 0 │
│ 1  │ 1  │ 0  │ 1  │ 0 │
│ 1  │ 1  │ 1  │ 0  │ 1 │
│ 1  │ 1  │ 1  │ 1  │ 0 │
└────┴────┴────┴────┴───┘

Кофакторы и производные:
┌────┬────┬────┬─────┬─────┬────────┐
│ x₂ │ x₃ │ x₄ │ f₁₀ │ f₁₁ │ ∂f/∂x₁ │
├────┼────┼────┼─────┼─────┼────────┤
│ 0  │ 0  │ 0  │  0  │  0  │   0    │
│ 0  │ 0  │ 1  │  0  │  0  │   0    │
│ 0  │ 1  │ 0  │  1  │  1  │   0    │
│ 0  │ 1  │ 1  │  1  │  1  │   0    │
│ 1  │ 0  │ 0  │  0  │  0  │   0    │
│ 1  │ 0  │ 1  │  1  │  0  │   

In [ ]:
SUBS = {
    "0": "₀",
    "1": "₁",
    "2": "₂",
    "3": "₃",
    "4": "₄",
    "5": "₅",
    "6": "₆",
    "7": "₇",
    "8": "₈",
    "9": "₉",
}


def subscript(num):
    return "".join(SUBS[d] for d in str(num))


def varname(i):
    return f"x{subscript(i)}"


def box_table(headers, rows):
    widths = [
        max(len(str(h)), *(len(str(r[i])) for r in rows)) for i, h in enumerate(headers)
    ]

    def hline(left, mid, right, fill="─"):
        s = left
        for i, w in enumerate(widths):
            s += fill * (w + 2)
            s += mid if i < len(widths) - 1 else right
        return s

    top = hline("┌", "┬", "┐")
    header_sep = hline("├", "┼", "┤")
    bottom = hline("└", "┴", "┘")
    out = [top]
    hdr = "│"
    for i, h in enumerate(headers):
        hdr += " " + str(h).center(widths[i]) + " │"
    out.append(hdr)
    out.append(header_sep)
    for r in rows:
        row = "│"
        for i, c in enumerate(r):
            row += " " + str(c).center(widths[i]) + " │"
        out.append(row)
    out.append(bottom)
    return "\n".join(out)


def truth_table(n, ones):
    return [1 if i in ones else 0 for i in range(2**n)]


def check_P0(values):
    return values[0] == 0


def check_P1(values):
    return values[-1] == 1


def check_S(values):
    N = len(values)
    for i in range(N):
        if values[i] != 1 - values[N - 1 - i]:
            return False
    return True


def check_M(values):
    N = len(values)
    for i in range(N):
        for j in range(N):
            if (i & ~j) == 0 and values[i] > values[j]:
                return False
    return True


def pascal_triangle(values):
    rows = [values[:]]
    while len(rows[-1]) > 1:
        prev = rows[-1]
        nxt = [(prev[k] ^ prev[k + 1]) for k in range(len(prev) - 1)]
        rows.append(nxt)
    return rows


def check_L(values):
    rows = pascal_triangle(values)
    coeffs = [row[0] for row in rows]
    for idx, c in enumerate(coeffs):
        if c == 1 and (idx & (idx - 1)) != 0:
            return False
    return True


def main(ones):
    n = max(ones).bit_length()
    values = truth_table(n, ones)

    headers = [varname(i) for i in range(1, n + 1)] + ["f", "f*"]
    rows = []
    for m in range(2**n):
        bits = [(m >> (n - 1 - i)) & 1 for i in range(n)]
        rows.append(bits + [values[m], 1 - values[2**n - 1 - m]])
    print("Таблица истинности:")
    print(box_table(headers, rows))
    print()

    print(
        box_table(
            ["P₀", "P₁", "S", "L", "M"],
            [
                [
                    check_P0(values),
                    check_P1(values),
                    check_S(values),
                    check_L(values),
                    check_M(values),
                ]
            ],
        )
    )

    print("Треугольник Паскаля:")
    rows = pascal_triangle(values)
    maxlen = max(len(r) for r in rows)
    box_rows = []
    for r in rows:
        box_rows.append([str(x) for x in r] + [""] * (maxlen - len(r)))
    headers = [f"{i}" for i in range(maxlen)]
    print(box_table(headers, box_rows))


if __name__ == "__main__":
    main(list(map(int, input("Введите термы через пробел: ").split())))

Множество единичных наборов: [2, 3, 5, 7, 10, 11, 14]
Число переменных: n = 4

Таблица истинности:
┌────┬────┬────┬────┬───┬────┐
│ x₁ │ x₂ │ x₃ │ x₄ │ f │ f* │
├────┼────┼────┼────┼───┼────┤
│ 0  │ 0  │ 0  │ 0  │ 0 │ 1  │
│ 0  │ 0  │ 0  │ 1  │ 0 │ 0  │
│ 0  │ 0  │ 1  │ 0  │ 1 │ 1  │
│ 0  │ 0  │ 1  │ 1  │ 1 │ 1  │
│ 0  │ 1  │ 0  │ 0  │ 0 │ 0  │
│ 0  │ 1  │ 0  │ 1  │ 1 │ 0  │
│ 0  │ 1  │ 1  │ 0  │ 0 │ 1  │
│ 0  │ 1  │ 1  │ 1  │ 1 │ 1  │
│ 1  │ 0  │ 0  │ 0  │ 0 │ 0  │
│ 1  │ 0  │ 0  │ 1  │ 0 │ 1  │
│ 1  │ 0  │ 1  │ 0  │ 1 │ 0  │
│ 1  │ 0  │ 1  │ 1  │ 1 │ 1  │
│ 1  │ 1  │ 0  │ 0  │ 0 │ 0  │
│ 1  │ 1  │ 0  │ 1  │ 0 │ 0  │
│ 1  │ 1  │ 1  │ 0  │ 1 │ 1  │
│ 1  │ 1  │ 1  │ 1  │ 0 │ 1  │
└────┴────┴────┴────┴───┴────┘

┌──────┬───────┬───────┬───────┬───────┐
│  P₀  │   P₁  │   S   │   L   │   M   │
├──────┼───────┼───────┼───────┼───────┤
│ True │ False │ False │ False │ False │
└──────┴───────┴───────┴───────┴───────┘
Треугольник Паскаля:
┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬────┬────┬───

In [5]:
from collections import defaultdict

SUBS = {
    "0": "₀",
    "1": "₁",
    "2": "₂",
    "3": "₃",
    "4": "₄",
    "5": "₅",
    "6": "₆",
    "7": "₇",
    "8": "₈",
    "9": "₉",
}


def subscript(num):
    return "".join(SUBS[d] for d in str(num))


def varname(i):
    return f"x{subscript(i)}"


def box_table(headers, rows):
    widths = [
        max(len(str(h)), *(len(str(r[i])) for r in rows)) for i, h in enumerate(headers)
    ]

    def hline(left, mid, right, fill="─"):
        s = left
        for i, w in enumerate(widths):
            s += fill * (w + 2)
            s += mid if i < len(widths) - 1 else right
        return s

    top = hline("┌", "┬", "┐")
    header_sep = hline("├", "┼", "┤")
    bottom = hline("└", "┴", "┘")
    out = [top]
    hdr = "│"
    for i, h in enumerate(headers):
        hdr += " " + str(h).center(widths[i]) + " │"
    out.append(hdr)
    out.append(header_sep)
    for r in rows:
        row = "│"
        for i, c in enumerate(r):
            row += " " + str(c).center(widths[i]) + " │"
        out.append(row)
    out.append(bottom)
    return "\n".join(out)


def to_binary_str(n, width):
    return format(n, f"0{width}b")


def combine_terms(a, b):
    diff = 0
    result = ""
    for x, y in zip(a, b):
        if x != y:
            diff += 1
            result += "-"
        else:
            result += x
    return result if diff == 1 else None


def get_prime_implicants(terms, num_vars):
    minterms = [to_binary_str(m, num_vars) for m in terms]
    groups = {}
    for m in minterms:
        ones = m.count("1")
        groups.setdefault(ones, []).append(m)
    prime_implicants = set()
    while groups:
        new_groups = {}
        used = set()
        marked = set()
        sorted_keys = sorted(groups.keys())
        for i in range(len(sorted_keys) - 1):
            group1 = groups[sorted_keys[i]]
            group2 = groups[sorted_keys[i + 1]]
            for a in group1:
                for b in group2:
                    combined = combine_terms(a, b)
                    if combined:
                        ones = combined.replace("-", "").count("1")
                        new_groups.setdefault(ones, []).append(combined)
                        used.add(combined)
                        marked.add(a)
                        marked.add(b)
        for group in groups.values():
            for term in group:
                if term not in marked:
                    prime_implicants.add(term)
        groups = new_groups
    return list(prime_implicants)


def covers(implicant, term):
    return all(ic == tc or ic == "-" for ic, tc in zip(implicant, term))


def generate_coverage_table(implicants, minterms, num_vars):
    table = defaultdict(set)
    binary_minterms = [to_binary_str(m, num_vars) for m in minterms]
    for imp in implicants:
        for m_bin, m_dec in zip(binary_minterms, minterms):
            if covers(imp, m_bin):
                table[imp].add(m_dec)
    return table


def get_essential_implicants(prime_implicants, minterms, num_vars):
    table = generate_coverage_table(prime_implicants, minterms, num_vars)
    essential = set()
    covered = set()
    minterm_to_imps = defaultdict(set)
    for imp, mts in table.items():
        for m in mts:
            minterm_to_imps[m].add(imp)
    for m in minterms:
        if len(minterm_to_imps[m]) == 1:
            only_imp = next(iter(minterm_to_imps[m]))
            essential.add(only_imp)
    for imp in essential:
        covered.update(table[imp])
    remaining_minterms = set(minterms) - covered
    non_essential = set(prime_implicants) - essential
    while remaining_minterms:
        best = max(non_essential, key=lambda x: len(table[x] & remaining_minterms))
        essential.add(best)
        covered.update(table[best])
        remaining_minterms = set(minterms) - covered
        non_essential.remove(best)
    return essential


def literal_to_str(lit, var_names):
    result = []
    for b, v in zip(lit, var_names):
        if b == "1":
            result.append(v)
        elif b == "0":
            result.append("¬" + v)
    return "".join(result) if result else "1"


def print_as_dnf(var_names, terms):
    dnf = " ∨ ".join(literal_to_str(t, var_names) for t in terms)
    print("ДНФ: f =", dnf, "\n")


def print_coverage_table(var_names, coverage_table, implicants, minterms):
    headers = ["Импликанта"] + [str(m) for m in minterms]
    rows = []
    for imp in implicants:
        row = [literal_to_str(imp, var_names)]
        row += ["+" if m in coverage_table[imp] else "" for m in minterms]
        rows.append(row)
    print(box_table(headers, rows))
    print()


def main(minterms):
    num_vars = max(minterms).bit_length()
    var_names = [varname(i) for i in range(1, num_vars + 1)]

    dnf_terms = [to_binary_str(m, num_vars) for m in minterms]
    print_as_dnf(var_names, dnf_terms)

    prime_implicants = get_prime_implicants(minterms, num_vars)
    print("Простые импликанты:")
    print_as_dnf(var_names, prime_implicants)

    coverage_table = generate_coverage_table(prime_implicants, minterms, num_vars)
    print("Таблица покрытия:")
    print_coverage_table(var_names, coverage_table, prime_implicants, minterms)

    essential_implicants = get_essential_implicants(
        prime_implicants, minterms, num_vars
    )
    print("Минимальное покрытие:")
    print_as_dnf(var_names, essential_implicants)
    print("Таблица минимального покрытия:")
    print_coverage_table(var_names, coverage_table, essential_implicants, minterms)

    return essential_implicants


if __name__ == "__main__":
    main(list(map(int, input("Введите термы через пробел: ").split())))

ДНФ: f = ¬x₁¬x₂¬x₃x₄ ∨ ¬x₁x₂¬x₃¬x₄ ∨ ¬x₁x₂¬x₃x₄ ∨ ¬x₁x₂x₃x₄ ∨ x₁x₂¬x₃¬x₄ ∨ x₁x₂x₃x₄ 

Простые импликанты:
ДНФ: f = ¬x₁x₂x₄ ∨ ¬x₁x₂¬x₃ ∨ x₂x₃x₄ ∨ ¬x₁¬x₃x₄ ∨ x₂¬x₃¬x₄ 

Таблица покрытия:
┌────────────┬───┬───┬───┬───┬────┬────┐
│ Импликанта │ 1 │ 4 │ 5 │ 7 │ 12 │ 15 │
├────────────┼───┼───┼───┼───┼────┼────┤
│  ¬x₁x₂x₄   │   │   │ + │ + │    │    │
│  ¬x₁x₂¬x₃  │   │ + │ + │   │    │    │
│   x₂x₃x₄   │   │   │   │ + │    │ +  │
│  ¬x₁¬x₃x₄  │ + │   │ + │   │    │    │
│  x₂¬x₃¬x₄  │   │ + │   │   │ +  │    │
└────────────┴───┴───┴───┴───┴────┴────┘

Минимальное покрытие:
ДНФ: f = x₂x₃x₄ ∨ ¬x₁¬x₃x₄ ∨ x₂¬x₃¬x₄ 

Таблица минимального покрытия:
┌────────────┬───┬───┬───┬───┬────┬────┐
│ Импликанта │ 1 │ 4 │ 5 │ 7 │ 12 │ 15 │
├────────────┼───┼───┼───┼───┼────┼────┤
│   x₂x₃x₄   │   │   │   │ + │    │ +  │
│  ¬x₁¬x₃x₄  │ + │   │ + │   │    │    │
│  x₂¬x₃¬x₄  │   │ + │   │   │ +  │    │
└────────────┴───┴───┴───┴───┴────┴────┘

